In [19]:
from tree_sitter import Language, Parser

Language.build_library(
  # Store the library in the `build` directory
  'build/my-languages.so',

  # Include one or more languages
  [
    #'Somenotebooks/tsitter/tree-sitter-python'
    #'/home/aktarafder/Documents/SoftwareProject/HF/Somenotebooks/tsitter/tree-sitter-python',
    './tree-sitter-python'
    #'/home/aktarafder/Documents/SoftwareProject/HF/Somenotebooks/tsitter/tree-sitter-java'
  ]
)
PYTHON_LANGUAGE = Language('build/my-languages.so', 'python')
parser = Parser()
parser.set_language(PYTHON_LANGUAGE)


In [20]:
import torch
from torch.utils.data.dataset import Dataset
import transformers
import tokenizers
from tokenizers import Tokenizer, pre_tokenizers
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace, Split
import json
from datasets import load_dataset, load_dataset_builder, get_dataset_split_names,get_dataset_config_names
from transformers import PreTrainedTokenizerFast
from transformers import DataCollatorForLanguageModeling
from transformers import BertConfig, BertLMHeadModel
from transformers import Trainer, TrainingArguments
from tokenizers import ByteLevelBPETokenizer

import tree_sitter # parsing library
from tree_sitter import Language, Parser
from datasets import load_dataset
import pathlib
import re
from collections import defaultdict

# loading codesearch net java dataset
dataset_codeSearch_python=load_dataset("code_search_net","python")
dpython=dataset_codeSearch_python
print("loaded the dataset.")
unk_token = "<UNK>"  # token for unknown words


Found cached dataset code_search_net (/home/aktarafder/.cache/huggingface/datasets/code_search_net/python/1.0.0/8f2524e6b62f65af5f5d65c53715c654db7b08dc93e0b7bcce2ab2f286a75be1)
100%|██████████| 3/3 [00:00<00:00, 171.02it/s]

loaded the dataset.


build an initial corpus

In [21]:
all_tokens_from_ast=[]

with open('./TXT_Files/tokens1to25k.txt','r') as f:
    contents=f.readlines()
all_tokens_from_ast = [line.strip() for line in contents]
token_set=set(all_tokens_from_ast)
all_tokens_from_ast.reverse()

In [22]:
def getFreq(token_list):
    word_freq=defaultdict(int)
    for token in token_list:
        word_freq[token]+=1
    
    return word_freq

def topK(word_freq,K):
    s=sorted(word_freq.items(), key= lambda x:x[1], reverse=True)
    return defaultdict(int,s[:K]) # assign

In [23]:
fr=getFreq(all_tokens_from_ast)
s=topK(fr,20000)
vocab = list()
vocab.extend(s.keys())

In [24]:
def calPFreq(splits):

    pair_freqs=defaultdict(int)
    for j in range(len(splits)-1):
        pair=(splits[j], splits[j+1])
        if (splits[j].startswith('#') or splits[j+1].startswith('#')):
            continue
        else:
            pair_freqs[pair]+=1

    return pair_freqs

In [25]:
pair_freq=calPFreq(all_tokens_from_ast)

In [26]:
best_pair = ""
max_freq = None

for pair, freq in pair_freq.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print(best_pair, max_freq)

('self', '.') 52926


In [27]:
def merge_pair(a, b, splits):
    #for word in splits:
    i = 0
    while i < len(splits) - 1:
        if splits[i] == a and splits[i + 1] == b:
            if len(a)==1 or len(b)==1:
                splits = splits[:i] + [a + b] + splits[i + 2 :]
            else:
                splits = splits[:i] + [a +" "+b] + splits[i + 2 :]
        else:
            i += 1
    #splits[word] = splits
    return splits

In [28]:
len(vocab)

20000

In [29]:
token_list=all_tokens_from_ast[:20000]

In [30]:
splits=merge_pair('self','.', token_list)

In [33]:
merge_size = 2000

splits=all_tokens_from_ast[:30000]
merges=dict()
for i in range(merge_size):
    pair_freqs = calPFreq(splits)
    #print(pair_freqs)
    best_pair = next(iter(pair_freq))# just selecting the first one, so that no error shows later
    max_freq = None
    for pair, freq in pair_freqs.items():
        if (max_freq is None or max_freq < freq):
            best_pair = pair
            max_freq = freq
    #print('best pair:',best_pair)
    if max_freq>2:
        splits = merge_pair(best_pair[0], best_pair[1], splits)
        merges[best_pair] = best_pair[0] + best_pair[1]
        vocab.append(best_pair[0] + best_pair[1])
        #print(best_pair[0]+best_pair[1])

In [34]:
len(vocab)

22796

In [35]:
def assignEmbed(vocab):
    uniqId=0
    BPE=dict()
    for i in range(len(vocab)):
        BPE[vocab[i]]=uniqId
        uniqId=uniqId+1
    
    return BPE

In [36]:
bpe=assignEmbed(vocab)

In [38]:
import re
import multiprocessing

#merged_tokens=[]
def tokenizeSinlge(i):
    code_string=dpython['train']['whole_func_string'][i]
    split_text = re.split(r'(\s+)', code_string)

    #print(split_text)
    for pair, merge in merges.items():
        for idx, split in enumerate(split_text):
            j = 0
            while j < len(split) - 1:
                if split[j] == pair[0] and split[j + 1] == pair[1]:
                    #print("merge:", merge)
                    split_text= split_text[:j] + [merge] + split_text[j + 2 :]
                    
                j += 1
        # I need to have a dictionary update here, may be
    #merged_tokens.append(split_text)

    return split_text#sum(splits, [])

N=[i for i in range(20000)]

with multiprocessing.Pool() as pool:
    # Apply the process_item function to each item in the list
    # The map method distributes the workload across multiple processes
    mergedParallel = pool.map(tokenizeSinlge, N)


#print(merged_tokens==mergedParallel)


In [39]:
len(mergedParallel)

20000

In [180]:
multiprocessing.cpu_count()

128

In [40]:
len(bpe.keys())

21395

In [41]:
bpe_str = str(bpe)

with open('./TXT_ast_augmented_new.txt', 'w') as file:
    file.write(bpe_str)